### Import libraries and define node class

In [38]:
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
import random
import statistics

class ZollmanNode:
    def __init__(self, beliefProb, pEH=None):
        self.beliefProb = beliefProb
        self.pEH = pEH

In [104]:
# Create a random graph
globalVar = 10
graph = nx.erdos_renyi_graph(10, 0.3)


# Define interactive widgets for parameters
parameter_widget = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Parameter:')


# Define the function for one timestep
def testing(graph, parameter):
    # Update the graph based on the parameter
    # This is a placeholder function, you should replace it with your own logic
    print(graph, parameter)
    pass

# Define a function to update the graph based on the parameter value
def update_graph(p1, parameter):
    testing(graph, parameter)
    print(parameter)
    nx.draw(graph, with_labels=True)
    plt.show()

# Create a function to start the simulation
def start_simulation():
    widgets.interact_manual(update_graph, p1=10, parameter=parameter_widget)

# Display the widgets
start_simulation()


interactive(children=(IntSlider(value=10, description='p1', max=30, min=-10), FloatSlider(value=0.5, descripti…

### Define Function to Display Graph

### Define graph initialization function

In [106]:
def initialize_graph(type: str, numberNodes: int = 10):
    graph = nx.Graph()
    if type == 'circle':
        for index in range(numberNodes):
            initial_data = {'group': index % 4, 'beliefProb': random.random(), 'pEH': None}
            graph.add_node(f"scientist-{index}", **initial_data)
            if index > 0:
                graph.add_edge(f"scientist-{index - 1}", f"scientist-{index}")
        graph.add_edge(f"scientist-{numberNodes - 1}", "scientist-0")
    elif type == 'wheel':
        for index in range(numberNodes - 1):
            initial_data = {'group': index % 4, 'beliefProb': random.random(), 'pEH': None}
            graph.add_node(f"scientist-{index}", **initial_data)
            if index > 0:
                graph.add_edge(f"scientist-{index - 1}", f"scientist-{index}")
        graph.add_edge(f"scientist-{numberNodes - 2}", "scientist-0")
        graph.add_node(f"scientist-{numberNodes - 1}", group=1, beliefProb=random.random(), pEH=None)
        for node in graph.nodes:
            if node != f"scientist-{numberNodes - 1}":
                graph.add_edge(node, f"scientist-{numberNodes - 1}")
    else:
        for index in range(numberNodes):
            initial_data = {'group': index % 4, 'beliefProb': random.random(), 'pEH': None}
            graph.add_node(f"scientist-{index}", **initial_data)
        for i in range(numberNodes):
            for j in range(i + 1, numberNodes):
                if i != j:
                    graph.add_edge(f"scientist-{i}", f"scientist-{j}")
    return graph

### Define timestep function and helper functions

In [107]:
def run_experiment(world):
    return random.uniform(0.3, 0.8) if world == 'higher' else random.uniform(0.2, 0.7)

def calculate_posterior(pH, pEH):
    pE = pEH * pH + (1 - pEH) * (1 - pH)
    return (pEH * pH) / pE

def timestep(graph):
    nodes = graph.nodes(data=True)
    for node, data in nodes:
        data['pEH'] = run_experiment('higher') if data['beliefProb'] > 0.5 else None
        data['beliefProb'] = calculate_posterior(data['beliefProb'], data['pEH']) if data['pEH'] is not None else data['beliefProb']

    posteriors = []
    for node, data in nodes:
        neighbors = graph.neighbors(node)
        for neighbor in neighbors:
            neighbor_data = graph.nodes[neighbor]
            if neighbor_data['pEH'] is not None:
                data['beliefProb'] = calculate_posterior(data['beliefProb'], neighbor_data['pEH'])
        data['group'] = sum(data['beliefProb'] > x for x in [0, 0.4, 0.6, 1])
        posteriors.append(data['beliefProb'])
    return posteriors

### Define Function to Run Simulation

In [108]:
def run_simulation(graph, num_timesteps=1):
    median_posteriors = []
    for _ in range(num_timesteps):
        posteriors = timestep(graph)
        median_posterior = statistics.median(posteriors)
        median_posteriors.append(median_posterior if median_posterior is not None else 0)
    return median_posteriors

### Run Simulation and Display Results

In [109]:
# Example usage:
graph = initialize_graph('circle', numberNodes=10)
run_simulation(graph, num_timesteps=5)
plot_network(graph)

NameError: name 'random' is not defined